In [1]:
################################################
# intialize glove
################################################
# >pip install glove_python
# >wget http://www.google.com/search?q=glove+word+embeddings+download+those+dope+pre+trained+vectors
# >unzip dope_glove_shit.zip
# >cp cmps242_hw5_config.py.example cmps242_hw5_config.py
# >echo "set GLOVE_LOCATION in cmps242_hw5_config.py to one of those files"
################################################
from cmps242_hw5_config import *
from glove import Glove

glove_data = Glove.load_stanford(GLOVE_LOCATION)

print(glove_data.most_similar('test', number=10))

[('tests', 0.9164356610046378), ('testing', 0.81992214225169979), ('tested', 0.74428756870386203), ('final', 0.69102279899050834), ('taking', 0.68790020450861178), ('results', 0.68468909013686308), ('match', 0.67696818430673733), ('determine', 0.67679768970507126), ('challenge', 0.67477055567439548)]


In [2]:
################################################
# file parsing functions
################################################
from nltk.tokenize import TweetTokenizer
import string, re
import collections
import numpy as np
import glove
from glove.glove_cython import fit_vectors, transform_paragraph

# definitions
HC="HillaryClinton"
DT="realDonaldTrump"
NA="none"
HANDLES = [HC,DT,NA]
HANDLE_MAP = {NA:0, HC:1, DT:2}

# read csv file, return handles and tweets
def parse_tweet_csv(file, file_encoding="utf8"):
    # init
    handles, tweets = [], []
    
    # read file
    linenr = -1
    with open(file, encoding=file_encoding) as input:
        try:
            for line in input:
                linenr += 1
                if linenr == 0: continue
                
                # get contents
                line = line.split(",")
                if line[0] in HANDLES: #label and irst line of tweet
                    handles.append(line[0])
                    tweet = ','.join(line[1:])
                    tweets.append(tweet)
                else: #second+ line of tweet
                    tweet = tweets.pop()
                    tweet += ','.join(line)
                    tweets.append(tweet)
        except Exception as e:
            print("Exception at line {}: {}".format(linenr, e))
            raise e
    
    # sanity checks
    assert len(handles) == len(tweets)
    print("Found {} tweets in {} lines".format(len(tweets), linenr + 1))
    
    # return data
    return handles, tweets


##########################################
### coverting tweet strings to numbers ###

# coverting labels to integers
def int_labels(labels):
    return list(map(lambda x: HANDLE_MAP[x], labels))

#tokenizing
_tokenizer = TweetTokenizer()
_punctuation = set(string.punctuation)
def tokenize(tweet, lowercase=True, strip_urls=True, strip_punctuation=True):
    tokens = _tokenizer.tokenize(tweet)
    if lowercase: tokens = list(map(lambda x: x.lower(), tokens))
    if strip_urls: tokens = list(filter(lambda x: not x.startswith("http"), tokens))
    if strip_punctuation: #https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
        tokens = list(filter(lambda x: x.startswith(u'@') or x.startswith(u'#') or x not in _punctuation and not re.match(u"[^\w\d'\s$]+", x), tokens))
    return tokens

# glove information
def get_glove_vector(glove_data, tokens, epochs=50, ignore_missing=True):
    """
    tpesout: This code came from the 'glove' repo I'm using (but had a bug, so I needed to slighly modify it)
    https://github.com/maciejkula/glove-python/blob/master/glove/glove.py
    
    Transform an iterable of tokens into its vector representation
    (a paragraph vector).

    Experimental. This will return something close to a tf-idf
    weighted average of constituent token vectors by fitting
    rare words (with low word bias values) more closely.
    """

    if glove_data.word_vectors is None:
        raise Exception('Model must be fit to transform paragraphs')

    if glove_data.dictionary is None:
        raise Exception('Dictionary must be provided to '
                        'transform paragraphs')

    cooccurrence = collections.defaultdict(lambda: 0.0)

    for token in tokens:
        try:
            cooccurrence[glove_data.dictionary[token]] += glove_data.max_count / 10.0
        except KeyError:
            if not ignore_missing:
                raise

    random_state = glove.glove.check_random_state(glove_data.random_state)

    word_ids = np.array(list(cooccurrence.keys()), dtype=np.int32)
    values = np.array(list(cooccurrence.values()), dtype=np.float64)
    shuffle_indices = np.arange(len(word_ids), dtype=np.int32)

    # Initialize the vector to mean of constituent word vectors
    paragraph_vector = np.mean(glove_data.word_vectors[word_ids], axis=0)
    sum_gradients = np.ones_like(paragraph_vector)

    # Shuffle the coocurrence matrix
    random_state.shuffle(shuffle_indices)
    transform_paragraph(glove_data.word_vectors,
                        glove_data.word_biases,
                        paragraph_vector,
                        sum_gradients,
                        word_ids,
                        values,
                        shuffle_indices,
                        glove_data.learning_rate,
                        glove_data.max_count,
                        glove_data.alpha,
                        epochs)

    return paragraph_vector
    
    
# get all tweets
def import_text(tweets):
    return [get_glove_vector(glove_data, tokenize(tweet)) for tweet in tweets]

In [3]:
################################################
# get raw test data
################################################
import random

# init
TEST_RATIO = 0.1
assert TEST_RATIO > 0 and TEST_RATIO < 1

# get data
text_handles, raw_tweets = parse_tweet_csv("train.csv")
handles = int_labels(text_handles)
tweets = import_text(raw_tweets)   
data_vector_size = len(tweets[0])

### validation
for i in range(1):
    tweet = raw_tweets[random.randint(0, len(raw_tweets))]
    print(tokenize(tweet))
    print(get_glove_vector(glove_data, tokenize(tweet)))
    print()
# for handle in int_labels(handles[0:7]):
#     print(handle)

Found 4743 tweets in 6251 lines


C:\Program Files\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Program Files\Anaconda3\lib\site-packages\numpy\core\_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


['#makeamericagreatagain', '#trump2016']
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan]



In [41]:
################################################
# split test data into train and test
################################################
import pandas as pd

LABEL = 'handle'
DATA = 'tweet_data'
LENGTH = 'length'

# this is so the glove output is in integer form (maybe not necessary)
FLOAT_GRANULARITY = (1 << 16)
VOCAB_SIZE = 2 * FLOAT_GRANULARITY # for positive and negative

# split into test and train
train_labels, train_data, test_labels, test_data = list(), list(), list(), list()
for handle, tweet in zip(handles, tweets):
    if np.isnan(tweet[0]): continue #a row of all nan's happens with data that glove can't understand (like, all hashtags)
    tweet = list(map(lambda x: int(x * FLOAT_GRANULARITY), tweet))
    if random.random() < TEST_RATIO:
        test_labels.append(handle)
        test_data.append(tweet)
    else:
        train_labels.append(handle)
        train_data.append(tweet)

# document and validate
print("Separated into {} train and {} test ({}%)\n".format(len(train_data), len(test_data), 
                                                         int(100.0 * len(test_data) / len(raw_tweets))))
assert len(train_labels) == len(train_data) and len(train_data) > 0
assert len(test_labels) == len(test_data) and len(test_data) > 0
assert len(test_labels) > len(tweets) * (TEST_RATIO - .05)
assert len(test_labels) < len(tweets) * (TEST_RATIO + .05) 

# save to dataframe
train = pd.DataFrame({
    LABEL: train_labels,
    DATA: train_data,
    LENGTH: [data_vector_size for _ in range(len(train_data))]
})
test = pd.DataFrame({
    LABEL: test_labels,
    DATA: test_data,
    LENGTH: [data_vector_size for _ in range(len(test_data))]
})
print(train.head())

Separated into 4229 train and 453 test (9%)

   handle  length                                         tweet_data
0       1      50  [-42109, -10098, 4462, 15191, -31174, -22236, ...
1       1      50  [-30288, -6132, -17246, 20218, -20686, -8332, ...
2       1      50  [-36519, 21300, -29862, 33744, -21948, 13709, ...
3       1      50  [-22020, -24918, -1488, 9447, -33670, -27225, ...
4       2      50  [-31240, -19811, 12546, -4235, -42001, -16107,...


In [42]:
################################################
# initializing our tensor
#
# based off of blogpost david parks showed us:
# https://r2rt.com/recurrent-neural-networks-in-tensorflow-iii-variable-length-sequences.html
################################################
import tensorflow as tf
 
 
class DataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n-1 > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n
        # return res['as_numbers'], res['gender']*3 + res['age_bracket'], res['length']
        # return res[DATA], res[LABEL], res[LENGTH]
        return res[DATA], res[LABEL], res[LENGTH]


def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()


def build_graph(vocab_size = VOCAB_SIZE, state_size = 64, batch_size = 16, num_classes = 2):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.constant(1.0)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.GRUCell(state_size)
    init_state = tf.get_variable('init_state', [1, state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state)

    # Add dropout, as the model otherwise quickly overfits
    rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)
    idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }


def train_graph(g, batch_size = 256, num_epochs = 10, iterator = DataIterator):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        tr = iterator(train)
        te = iterator(test)

        step, accuracy = 0, 0
        tr_losses, te_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
            print(feed)
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                te_epoch = te.epochs
                while te.epochs == te_epoch:
                    step += 1
                    batch = te.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                te_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- te:", te_losses[-1])

    return tr_losses, te_losses

In [43]:
################################################
# explore our data iterator
################################################

# validate data iterator
d = DataIterator(test).next_batch(3)
print('Input sequences:\n', 
      "{}: \n{}\n".format(type(d[0]), d[0]), 
      "{}: \n{}\n".format(type(d[0][0]), d[0][0]), 
      "{}: \n{}\n".format(type(d[0][0][0]), d[0][0][0]), 
      end='\n\n')
print('Target values\n', 
      "{}: \n{}\n".format(type(d[1]), d[1]), 
      "{}: \n{}\n".format(type(d[1][0]), d[1][0]), 
      end='\n\n')
print('Sequence lengths\n', 
      "{}: \n{}\n".format(type(d[2]), d[2]), 
      "{}: \n{}\n".format(type(d[2][0]), d[2][0]), 
      end='\n\n')

Input sequences:
 <class 'pandas.core.series.Series'>: 
0    [-23220, -16038, 21501, 19702, -29209, -18200,...
1    [-29662, -30123, 9171, -12641, -43482, -34101,...
2    [-40992, -34769, 8202, 4369, -38950, -23321, 3...
Name: tweet_data, dtype: object
 <class 'list'>: 
[-23220, -16038, 21501, 19702, -29209, -18200, 20775, 15318, 113, 679, -8878, 25544, 18791, 22421, -12248, 5307, 6147, -705, 29726, 27388, -18505, 314, -11106, 15874, -17429, -19174, 21538, -24105, -32373, -15124, 84529, 18018, 24098, 33385, 13006, 9015, -820, -15473, -24512, 2098, 8037, -11588, 24353, 12246, 10620, -1562, -4760, -18723, -12746, 14142]
 <class 'int'>: 
-23220


Target values
 <class 'pandas.core.series.Series'>: 
0    1
1    1
2    1
Name: handle, dtype: int64
 <class 'numpy.int64'>: 
1


Sequence lengths
 <class 'pandas.core.series.Series'>: 
0    50
1    50
2    50
Name: length, dtype: int64
 <class 'numpy.int64'>: 
50




In [44]:
################################################
# run it!
################################################

# this fails, just like us
g = build_graph()
tr_losses, te_losses = train_graph(g)


C:\Program Files\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


{<tf.Tensor 'Placeholder:0' shape=(16, ?) dtype=int32>: 0      [-19597, 3807, -8248, 11081, -36083, 16265, 17...
1      [-7391, -22530, -6053, 31666, -11602, -8017, 1...
2      [-24991, -20675, -9795, 33102, -30620, -30586,...
3      [-29375, -18296, 33733, -2698, -32659, -27741,...
4      [-37291, -26696, -12705, 16690, -34839, -26794...
5      [-11776, 10437, -20617, 16236, -5312, -9818, 1...
6      [11274, -25112, -15371, 15847, -7950, 13641, 7...
7      [-27797, -31908, 18046, 10629, -33857, 6124, 4...
8      [-11224, 6346, 2639, 30194, -17485, -31192, 30...
9      [-33930, -16429, -12657, 18986, -25771, -17956...
10     [4769, -17286, -15732, -8530, 31991, -5158, 11...
11     [-31725, 3431, -3099, 8552, -39946, -11157, 26...
12     [-28251, -13058, -10185, 12182, -17267, -13589...
13     [-19279, 3354, 13119, 14376, -22925, -16620, 5...
14     [-19903, -29001, -19340, 11088, -20989, 21317,...
15     [-39233, -21299, -8505, 11710, -23503, -27121,...
16     [-31199, -19418, 4889, -5

ValueError: setting an array element with a sequence.